In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [4]:
teams_df = pd.DataFrame()
i = 2010
while i <=2019:
    print("Downloading data for the", i, "season.")
    start = dt.datetime.now()
    teams = Teams(year = i)
    end = dt.datetime.now()
    teams_temp = teams.dataframes
    teams_temp['Season'] = i
    teams_df = pd.concat([teams_df, teams_temp])
    span = end - start
    print(i, "took", span, "seconds to download.")
    i += 1

2010 took 0:00:20.515166 seconds to download.
2011 took 0:00:20.536113 seconds to download.
2012 took 0:00:20.277801 seconds to download.
2013 took 0:00:20.556057 seconds to download.
2014 took 0:00:20.619887 seconds to download.
2015 took 0:00:20.638837 seconds to download.
2016 took 0:00:20.265835 seconds to download.
2017 took 0:00:20.450341 seconds to download.
2018 took 0:00:21.563367 seconds to download.
2019 took 0:00:22.422071 seconds to download.


In [6]:
teams_df.head()

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,...,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins,Season
AIR-FORCE,AIR-FORCE,61.6,389,10,0,6.2,51,mwc,15,1,...,0.527,456,46.8,851,0.529,19.0,394,0.323,10,2010
AKRON,AKRON,53.9,473,4,7,8.5,114,mac,4,12,...,0.483,642,51.6,1269,0.521,16.4,464,0.686,24,2010
ALABAMA-AM,ALABAMA-AM,48.1,288,9,3,12.7,143,swac,10,8,...,0.410,491,46.8,1004,0.463,18.8,444,0.407,11,2010
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,51.1,390,4,9,7.3,91,cusa,5,11,...,0.474,586,53.6,1248,0.518,17.0,451,0.735,25,2010
ALABAMA-STATE,ALABAMA-STATE,60.0,400,11,6,11.1,129,swac,6,12,...,0.448,477,51.3,1101,0.499,20.0,501,0.516,16,2010


In [124]:
teams_df.columns

Index(['abbreviation', 'assist_percentage', 'assists', 'away_losses',
       'away_wins', 'block_percentage', 'blocks', 'conference',
       'conference_losses', 'conference_wins', 'defensive_rebounds',
       'effective_field_goal_percentage', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'free_throws_per_field_goal_attempt', 'games_played', 'home_losses',
       'home_wins', 'losses', 'minutes_played', 'name', 'net_rating',
       'offensive_rating', 'offensive_rebound_percentage',
       'offensive_rebounds', 'opp_assist_percentage', 'opp_assists',
       'opp_block_percentage', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_effective_field_goal_percentage', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempt_rate', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_thro

In [135]:
from sportsreference.ncaab.schedule import Schedule

purdue_schedule = Schedule('ARIZONA')
for game in purdue_schedule:
    boxscore = game.boxscore
    print(game.boxscore_index)
#df = purdue_schedule.dataframe

2019-11-06-21-arizona
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [60]:
game_data = Boxscore('2019-11-06-19-purdue')
df = game_data.dataframe

In [63]:
cols = ['away_assist_percentage', 'away_assists', 'away_block_percentage',
       'away_blocks', 'away_defensive_rating',
       'away_defensive_rebound_percentage', 'away_defensive_rebounds',
       'away_effective_field_goal_percentage', 'away_field_goal_attempts',
       'away_field_goal_percentage', 'away_field_goals',
       'away_free_throw_attempt_rate', 'away_free_throw_attempts',
       'away_free_throw_percentage', 'away_free_throws', 'away_losses',
       'away_minutes_played', 'away_offensive_rating',
       'away_offensive_rebound_percentage', 'away_offensive_rebounds',
       'away_personal_fouls', 'away_points', 'away_ranking',
       'away_steal_percentage', 'away_steals', 'away_three_point_attempt_rate',
       'away_three_point_field_goal_attempts',
       'away_three_point_field_goal_percentage',
       'away_three_point_field_goals', 'away_total_rebound_percentage',
       'away_total_rebounds', 'away_true_shooting_percentage',
       'away_turnover_percentage', 'away_turnovers',
       'away_two_point_field_goal_attempts',
       'away_two_point_field_goal_percentage', 'away_two_point_field_goals',
       'away_win_percentage', 'away_wins', 'date', 'home_assist_percentage',
       'home_assists', 'home_block_percentage', 'home_blocks',
       'home_defensive_rating', 'home_defensive_rebound_percentage',
       'home_defensive_rebounds', 'home_effective_field_goal_percentage',
       'home_field_goal_attempts', 'home_field_goal_percentage',
       'home_field_goals', 'home_free_throw_attempt_rate',
       'home_free_throw_attempts', 'home_free_throw_percentage',
       'home_free_throws', 'home_losses', 'home_minutes_played',
       'home_offensive_rating', 'home_offensive_rebound_percentage',
       'home_offensive_rebounds', 'home_personal_fouls', 'home_points',
       'home_ranking', 'home_steal_percentage', 'home_steals',
       'home_three_point_attempt_rate', 'home_three_point_field_goal_attempts',
       'home_three_point_field_goal_percentage',
       'home_three_point_field_goals', 'home_total_rebound_percentage',
       'home_total_rebounds', 'home_true_shooting_percentage',
       'home_turnover_percentage', 'home_turnovers',
       'home_two_point_field_goal_attempts',
       'home_two_point_field_goal_percentage', 'home_two_point_field_goals',
       'home_win_percentage', 'home_wins', 'location', 'losing_abbr',
       'losing_name', 'pace', 'winner', 'winning_abbr', 'winning_name']

In [77]:
df = pd.DataFrame(columns=cols)
for team in Teams():
    abr = team.abbreviation
    schedule = Schedule(abr)
    for game in schedule:
        print(game.boxscore_index)
        df2 = Boxscore(game.boxscore_index).dataframe
        df = df.append(df2)
        df['game_id'] = game

2019-11-05-20-abilene-christian
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-07-21-air-force
2019-11-09-16-air-force
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-19-akron
2019-11-08-19-west-virginia
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-20-tennessee-state
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-07-20-troy
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
2019-11-05-20-gonzaga
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

KeyboardInterrupt: 

In [87]:
teams = Teams()

In [4]:
teams = Teams(year = 2019)
for team in teams:
    print(team.abbreviation)

ABILENE-CHRISTIAN
AIR-FORCE
AKRON
ALABAMA-AM
ALABAMA-BIRMINGHAM
ALABAMA-STATE
ALABAMA
ALBANY-NY
ALCORN-STATE
AMERICAN
APPALACHIAN-STATE
ARIZONA-STATE
ARIZONA
ARKANSAS-LITTLE-ROCK
ARKANSAS-PINE-BLUFF
ARKANSAS-STATE
ARKANSAS
ARMY
AUBURN
AUSTIN-PEAY
BALL-STATE
BAYLOR
BELMONT
BETHUNE-COOKMAN
BINGHAMTON
BOISE-STATE
BOSTON-COLLEGE
BOSTON-UNIVERSITY
BOWLING-GREEN-STATE
BRADLEY
BRIGHAM-YOUNG
BROWN
BRYANT
BUCKNELL
BUFFALO
BUTLER
CAL-POLY
CAL-STATE-BAKERSFIELD
CAL-STATE-FULLERTON
CAL-STATE-NORTHRIDGE
CALIFORNIA-BAPTIST
CALIFORNIA-DAVIS
CALIFORNIA-IRVINE
CALIFORNIA-RIVERSIDE
CALIFORNIA-SANTA-BARBARA
CALIFORNIA
CAMPBELL
CANISIUS
CENTRAL-ARKANSAS
CENTRAL-CONNECTICUT-STATE
CENTRAL-FLORIDA
CENTRAL-MICHIGAN
CHARLESTON-SOUTHERN
CHARLOTTE
CHATTANOOGA
CHICAGO-STATE
CINCINNATI
CITADEL
CLEMSON
CLEVELAND-STATE
COASTAL-CAROLINA
COLGATE
COLLEGE-OF-CHARLESTON
COLORADO-STATE
COLORADO
COLUMBIA
CONNECTICUT
COPPIN-STATE
CORNELL
CREIGHTON
DARTMOUTH
DAVIDSON
DAYTON
DELAWARE-STATE
DELAWARE
DENVER
DEPAUL
DETROIT-MERCY

In [139]:
for team in teams:
    if team.abbreviation == 'ARIZONA':
        print(team.abbreviation)

ARIZONA


In [88]:
pur = pd.DataFrame(columns=cols)

schedule = Schedule('PURDUE',year = 2018)
for game in purdue_schedule:
    print(game.boxscore_index)
    df2 = Boxscore(game.boxscore_index).dataframe
    pur = pur.append(df2)
    pur['game_id'] = game
    


2019-11-06-19-purdue
2019-11-09-19-purdue
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [79]:
pur.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
2019-11-06-19-purdue,52.2,12,2.4,1,109.7,64.3,18,0.421,63,0.365,...,25,1.0,1,"Mackey Arena, West Lafayette, Indiana",GREEN-BAY,Green Bay,72.2,Home,PURDUE,Purdue
2019-11-09-19-purdue,75.0,18,13.3,6,101.5,61.3,19,0.633,45,0.533,...,17,0.5,1,"Mackey Arena, West Lafayette, Indiana",PURDUE,Purdue,65.2,Away,TEXAS,Texas


In [15]:
teams = Teams(year = 2019)

In [148]:
dataset = pd.DataFrame()

In [ ]:
'''
    elif team.abbreviation == 'FAIRFIELD':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'BALL-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'APPALACHIAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'DAVIDSON':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VIRGINIA-TECH':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'ROBERT-MORRIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'FLORIDA-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MICHIGAN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MARYLAND':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'TEXAS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'NOTRE-DAME':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
        
    elif team.abbreviation == 'OHIO':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'BELMONT':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'IOWA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MICHIGAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'WISCONSIN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'RUTGERS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'INDIANA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'OHIO-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'PENN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'MINNESOTA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'NEBRASKA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'ILLINOIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'NORTHWESTERN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'OLD-DOMINION':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VILLANOVA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'TENNESSEE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df])
    
    elif team.abbreviation == 'VIRGINIA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        dataset = pd.concat([dataset, df]) '''




or  or (team.abbreviation == 'DAVIDSON') or (team.abbreviation == 'VIRGINIA-TECH') or
    (team.abbreviation == 'ROBERT-MORRIS') or (team.abbreviation == 'TEXAS') or (team.abbreviation == 'MICHIGAN') or
    (team.abbreviation == 'MARYLAND') or (team.abbreviation == 'NOTRE-DAME') or (team.abbreviation == 'FLORIDA-STATE') or
    (team.abbreviation == 'OHIO') or (team.abbreviation == 'BELMONT') or (team.abbreviation == 'IOWA') or
    (team.abbreviation == 'MICHIGAN-STATE') or (team.abbreviation == 'WISCONSIN') or (team.abbreviation == 'RUTGERS') or
    (team.abbreviation == 'INDIANA') or (team.abbreviation == 'OHIO-STATE') or (team.abbreviation == 'PENN-STATE') or
    (team.abbreviation == 'MINNESOTA') or (team.abbreviation == 'NEBRASKA') or (team.abbreviation == 'ILLINOIS') or
    (team.abbreviation == 'NORTHWESTERN') or (team.abbreviation == 'OLD-DOMINION') or (team.abbreviation == 'VILLANOVA') or
    (team.abbreviation == 'TENNESSEE') or (team.abbreviation == 'VIRGINIA')

In [41]:
dataset = pd.DataFrame()
lst = []
#teams = Teams(year = 2019)
for team in teams:
    if team.abbreviation == 'PURDUE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    elif team.abbreviation == 'FAIRFIELD':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
        
    elif team.abbreviation == 'BALL-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
        
    elif team.abbreviation == 'APPALACHIAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'DAVIDSON':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'VIRGINIA-TECH':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'ROBERT-MORRIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
        
    elif team.abbreviation == 'FLORIDA-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'MICHIGAN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'MARYLAND':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'TEXAS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
        
    elif team.abbreviation == 'NOTRE-DAME':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
        
    elif team.abbreviation == 'OHIO':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'BELMONT':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'IOWA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'MICHIGAN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'WISCONSIN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'RUTGERS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'INDIANA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'OHIO-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'PENN-STATE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'MINNESOTA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'NEBRASKA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'ILLINOIS':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'NORTHWESTERN':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'OLD-DOMINION':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'VILLANOVA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'TENNESSEE':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)
    
    elif team.abbreviation == 'VIRGINIA':
        dataset['team']=team.abbreviation
        print(team.abbreviation)
        df = team.schedule.dataframe_extended
        df['team']=team.abbreviation
        lst.append(df)

for i in lst:
    dataset = pd.concat([dataset,i])

APPALACHIAN-STATE
BALL-STATE
BELMONT
DAVIDSON
FAIRFIELD
FLORIDA-STATE
ILLINOIS
INDIANA
IOWA
MARYLAND
MICHIGAN-STATE
MICHIGAN
MINNESOTA
NEBRASKA
NORTHWESTERN
NOTRE-DAME
OHIO-STATE
OHIO
OLD-DOMINION
PENN-STATE
PURDUE
ROBERT-MORRIS
RUTGERS
TENNESSEE
TEXAS
VILLANOVA
VIRGINIA-TECH
VIRGINIA
WISCONSIN


C:\Users\bdraus\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:208: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [42]:
dataset

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,team,winner,winning_abbr,winning_name
2018-11-06-19-appalachian-state,31.8,7.0,2.4,1.0,152.4,57.1,16.0,0.427,55.0,0.400,...,1.000,1.0,"Holmes Convocation Center, Boone, North Carolina",Mars Hill\n\t\t\t,Mars Hill\n\t\t\t,82.4,APPALACHIAN-STATE,Home,APPALACHIAN-STATE,Appalachian State
2018-11-11-19-alabama,28.0,7.0,2.0,1.0,109.5,56.3,27.0,0.578,51.0,0.490,...,1.000,2.0,"Coleman Coliseum , Tuscaloosa, Alabama",APPALACHIAN-STATE,Appalachian State,74.2,APPALACHIAN-STATE,Home,ALABAMA,Alabama
2018-11-15-17-appalachian-state,44.4,16.0,17.9,5.0,94.6,80.0,24.0,0.644,66.0,0.545,...,0.333,1.0,"TD Arena, Charleston, South Carolina",APPALACHIAN-STATE,Appalachian State,74.1,APPALACHIAN-STATE,Away,PURDUE,Purdue
2018-11-16-16-wichita-state,53.8,14.0,5.1,2.0,117.1,65.5,19.0,0.573,55.0,0.473,...,0.500,2.0,"TD Arena, Charleston, South Carolina",APPALACHIAN-STATE,Appalachian State,69.7,APPALACHIAN-STATE,Home,WICHITA-STATE,Wichita State
2018-11-18-11-appalachian-state,57.1,16.0,11.5,6.0,95.6,73.2,30.0,0.517,59.0,0.475,...,0.200,1.0,"TD Arena, Charleston, South Carolina",APPALACHIAN-STATE,Appalachian State,79.7,APPALACHIAN-STATE,Away,BALL-STATE,Ball State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-07-19-wisconsin,55.6,10.0,11.8,4.0,95.6,59.5,22.0,0.347,59.0,0.305,...,0.700,21.0,"Kohl Center, Madison, Wisconsin",IOWA,Iowa,68.3,WISCONSIN,Home,WISCONSIN,Wisconsin
2019-03-10-16-ohio-state,48.1,13.0,7.3,3.0,91.8,82.5,33.0,0.469,65.0,0.415,...,0.581,18.0,"Value City Arena, Columbus, Ohio",OHIO-STATE,Ohio State,64.7,WISCONSIN,Away,WISCONSIN,Wisconsin
2019-03-15-14-wisconsin,29.2,7.0,22.6,7.0,100.0,66.7,14.0,0.444,63.0,0.381,...,0.719,23.0,"United Center, Chicago, Illinois",NEBRASKA,Nebraska,65.8,WISCONSIN,Home,WISCONSIN,Wisconsin
2019-03-16-13-michigan-state,37.5,9.0,15.6,5.0,98.5,76.0,19.0,0.368,68.0,0.353,...,0.818,27.0,"United Center, Chicago, Illinois",WISCONSIN,Wisconsin,68.2,WISCONSIN,Home,MICHIGAN-STATE,Michigan State


In [24]:
df = dataset.copy()

In [25]:
df.team.value_counts()

PURDUE    67
Name: team, dtype: int64

In [213]:
purdue.sort_values(by='date',inplace=True)

In [214]:
purdue['game_index'] = np.arange(len(purdue))

In [26]:
df

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,team,winner,winning_abbr,winning_name
2018-11-06-19-purdue,36.4,8.0,0.0,0.0,130.4,61.9,13.0,0.433,60.0,0.367,...,1.000,1.0,"Mackey Arena, West Lafayette, Indiana",FAIRFIELD,Fairfield,69.2,PURDUE,Home,PURDUE,Purdue
2018-11-10-19-bucknell,20.0,4.0,8.1,3.0,89.2,54.3,19.0,0.469,49.0,0.408,...,0.500,1.0,"Sojka Pavilion, Lewisburg, Pennsylvania",BUCKNELL,Bucknell,64.8,PURDUE,Away,FAIRFIELD,Fairfield
2018-11-13-19-fairfield,54.8,17.0,10.5,4.0,120.8,60.0,18.0,0.625,60.0,0.517,...,0.333,1.0,"Webster Bank Arena, Bridgeport, Connecticut",FAIRFIELD,Fairfield,71.7,PURDUE,Away,LONG-ISLAND-UNIVERSITY,LIU-Brooklyn
2018-11-17-13-fairfield,26.7,8.0,24.2,8.0,100.0,87.9,29.0,0.516,62.0,0.484,...,0.250,1.0,"Webster Bank Arena, Bridgeport, Connecticut",FAIRFIELD,Fairfield,73.1,PURDUE,Away,WAGNER,Wagner
2018-11-23-18-longwood,50.0,12.0,4.5,1.0,103.1,63.2,12.0,0.474,58.0,0.414,...,0.833,5.0,"Connolly Center, Seattle, Washington",FAIRFIELD,Fairfield,65.4,PURDUE,Home,LONGWOOD,Longwood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-15-19-purdue,59.3,16.0,9.5,4.0,110.6,65.7,23.0,0.546,54.0,0.500,...,0.719,23.0,"United Center, Chicago, Illinois",PURDUE,Purdue,66.4,PURDUE,Away,MINNESOTA,Minnesota
2019-03-21-21-purdue,22.2,4.0,21.7,5.0,88.4,88.5,23.0,0.313,67.0,0.269,...,0.727,24.0,"XL Center, Hartford, Connecticut",OLD-DOMINION,Old Dominion,68.7,PURDUE,Home,PURDUE,Purdue
2019-03-23-20-purdue,35.0,7.0,0.0,0.0,138.1,56.0,14.0,0.440,58.0,0.345,...,0.735,25.0,"XL Center, Hartford, Connecticut",VILLANOVA,Villanova,62.9,PURDUE,Home,PURDUE,Purdue
2019-03-28-19-tennessee,47.1,16.0,11.4,5.0,122.1,74.4,32.0,0.659,63.0,0.540,...,0.838,31.0,"KFC Yum! Center, Louisville, Kentucky",TENNESSEE,Tennessee,68.9,PURDUE,Away,PURDUE,Purdue


In [166]:
FIELDS_TO_DROP = ['away_points', 'home_points', 'date', 'location',
                  'losing_abbr', 'losing_name', 'winner', 'winning_abbr',
                  'winning_name', 'home_ranking', 'away_ranking','game_index']

In [167]:
X = purdue.drop(FIELDS_TO_DROP, 1).dropna().drop_duplicates()
y = purdue[['home_points', 'away_points']].values

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
parameters = {'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6}
model = RandomForestRegressor(**parameters)
model.fit(X_train, y_train)
print(model.predict(X_test).astype(int), y_test)

[[72 74]
 [71 72]
 [72 77]
 [74 66]
 [79 72]
 [90 64]
 [89 64]
 [78 59]
 [76 64]
 [79 70]] [[ 65  78]
 [ 66  75]
 [ 75  80]
 [ 68  65]
 [ 92  88]
 [111  42]
 [ 97  48]
 [ 82  51]
 [ 57  53]
 [ 82  75]]


In [169]:
preds = model.predict(X_test).astype(int), y_test

In [176]:
y_test

array([[ 65,  78],
       [ 66,  75],
       [ 75,  80],
       [ 68,  65],
       [ 92,  88],
       [111,  42],
       [ 97,  48],
       [ 82,  51],
       [ 57,  53],
       [ 82,  75]], dtype=int64)

In [180]:
purdue.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name,game_index
2017-12-01-19-maryland,63.0,17,20.0,7,111.9,65.9,27,0.594,53,0.509,...,0.667,6,"Xfinity Center, College Park, Maryland",MARYLAND,Maryland,66.7,Away,PURDUE,Purdue,0
2017-12-10-19-purdue,43.5,10,3.2,1,119.4,73.9,17,0.491,54,0.426,...,0.833,10,"Mackey Arena, West Lafayette, Indiana",IUPUI,IUPUI,72.2,Home,PURDUE,Purdue,1
2017-12-16-12-purdue,42.3,11,10.0,3,107.9,77.3,17,0.428,69,0.377,...,0.846,11,"Bankers Life Fieldhouse, Indianapolis, Indiana",BUTLER,Butler,76.1,Home,PURDUE,Purdue,2
2017-12-21-18-purdue,53.3,8,6.3,2,134.7,82.6,19,0.388,49,0.306,...,0.857,12,"Mackey Arena, West Lafayette, Indiana",TENNESSEE-STATE,Tennessee State,71.5,Home,PURDUE,Purdue,3
2017-12-03-16-purdue,56.7,17,11.4,4,117.5,71.0,22,0.524,63,0.476,...,0.800,8,"Mackey Arena, West Lafayette, Indiana",NORTHWESTERN,Northwestern,63.4,Home,PURDUE,Purdue,4


In [186]:
def get_home(row):
    
    if row.winner == 'Away':
        home = row.losing_abbr
        
    if row.winner == 'Home':
        home = row.winning_abbr
    return home

def get_away(row):
    
    if row.winner == 'Home':
        away = row.losing_abbr
        
    if row.winner == 'Away':
        away = row.winning_abbr
    return away

In [187]:
purdue['HomeTeam'] = purdue.apply (lambda row: get_home(row), axis=1)
purdue['AwayTeam'] = purdue.apply (lambda row: get_away(row), axis=1)

In [191]:
purdue

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,...,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name,game_index,HomeTeam,AwayTeam
2017-12-01-19-maryland,63.0,17,20.0,7,111.9,65.9,27,0.594,53,0.509,...,"Xfinity Center, College Park, Maryland",MARYLAND,Maryland,66.7,Away,PURDUE,Purdue,0,MARYLAND,PURDUE
2017-12-10-19-purdue,43.5,10,3.2,1,119.4,73.9,17,0.491,54,0.426,...,"Mackey Arena, West Lafayette, Indiana",IUPUI,IUPUI,72.2,Home,PURDUE,Purdue,1,PURDUE,IUPUI
2017-12-16-12-purdue,42.3,11,10.0,3,107.9,77.3,17,0.428,69,0.377,...,"Bankers Life Fieldhouse, Indianapolis, Indiana",BUTLER,Butler,76.1,Home,PURDUE,Purdue,2,PURDUE,BUTLER
2017-12-21-18-purdue,53.3,8,6.3,2,134.7,82.6,19,0.388,49,0.306,...,"Mackey Arena, West Lafayette, Indiana",TENNESSEE-STATE,Tennessee State,71.5,Home,PURDUE,Purdue,3,PURDUE,TENNESSEE-STATE
2017-12-03-16-purdue,56.7,17,11.4,4,117.5,71.0,22,0.524,63,0.476,...,"Mackey Arena, West Lafayette, Indiana",NORTHWESTERN,Northwestern,63.4,Home,PURDUE,Purdue,4,PURDUE,NORTHWESTERN
2017-12-30-20-purdue,62.5,15,2.5,1,136.1,57.1,20,0.468,62,0.387,...,"Mackey Arena, West Lafayette, Indiana",LIPSCOMB,Lipscomb,72.3,Home,PURDUE,Purdue,5,PURDUE,LIPSCOMB
2017-12-07-18-purdue,20.0,4,7.3,3,106.7,78.8,26,0.358,60,0.333,...,"Mackey Arena, West Lafayette, Indiana",VALPARAISO,Valparaiso,75.1,Home,PURDUE,Purdue,6,PURDUE,VALPARAISO
2018-02-10-16-michigan-state,61.5,16,7.1,3,106.3,79.3,23,0.453,64,0.406,...,"Breslin Events Center, East Lansing, Michigan",PURDUE,Purdue,63.8,Home,MICHIGAN-STATE,Michigan State,7,MICHIGAN-STATE,PURDUE
2018-02-15-19-wisconsin,36.8,7,10.0,3,87.7,82.9,29,0.438,48,0.396,...,"Kohl Center, Madison, Wisconsin",PURDUE,Purdue,65.3,Home,WISCONSIN,Wisconsin,8,WISCONSIN,PURDUE
2018-02-18-20-purdue,50.0,14,15.2,5,113.4,73.3,22,0.580,56,0.500,...,"Mackey Arena, West Lafayette, Indiana",PENN-STATE,Penn State,66.5,Home,PURDUE,Purdue,9,PURDUE,PENN-STATE


In [189]:
purdue.columns

Index(['away_assist_percentage', 'away_assists', 'away_block_percentage',
       'away_blocks', 'away_defensive_rating',
       'away_defensive_rebound_percentage', 'away_defensive_rebounds',
       'away_effective_field_goal_percentage', 'away_field_goal_attempts',
       'away_field_goal_percentage', 'away_field_goals',
       'away_free_throw_attempt_rate', 'away_free_throw_attempts',
       'away_free_throw_percentage', 'away_free_throws', 'away_losses',
       'away_minutes_played', 'away_offensive_rating',
       'away_offensive_rebound_percentage', 'away_offensive_rebounds',
       'away_personal_fouls', 'away_points', 'away_ranking',
       'away_steal_percentage', 'away_steals', 'away_three_point_attempt_rate',
       'away_three_point_field_goal_attempts',
       'away_three_point_field_goal_percentage',
       'away_three_point_field_goals', 'away_total_rebound_percentage',
       'away_total_rebounds', 'away_true_shooting_percentage',
       'away_turnover_percentage', 'awa

In [197]:
purdue.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 2017-12-01-19-maryland to 2017-11-28-20-purdue
Data columns (total 89 columns):
away_assist_percentage                    37 non-null float64
away_assists                              37 non-null int64
away_block_percentage                     37 non-null float64
away_blocks                               37 non-null int64
away_defensive_rating                     37 non-null float64
away_defensive_rebound_percentage         37 non-null float64
away_defensive_rebounds                   37 non-null int64
away_effective_field_goal_percentage      37 non-null float64
away_field_goal_attempts                  37 non-null int64
away_field_goal_percentage                37 non-null float64
away_field_goals                          37 non-null int64
away_free_throw_attempt_rate              37 non-null float64
away_free_throw_attempts                  37 non-null int64
away_free_throw_percentage                37 non-null float64
away_fr

In [208]:
def create_stats_features_ema(stats, span):
    
    # Create a copy of the DataFrame
    ema_features = stats[['team','winner', 'winning_abbr', 'winning_name','game_index', 'HomeTeam', 'AwayTeam','location','date','losing_abbr','losing_name']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = stats.drop(columns=['team','winner', 'winning_abbr', 'winning_name','game_index', 'HomeTeam', 'AwayTeam','location','date','losing_abbr','losing_name']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (stats.groupby('team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

In [209]:
stats = purdue.copy()
stats_features = create_stats_features_ema(stats, span=5)

In [210]:
stats_features

,team,winner,winning_abbr,winning_name,game_index,HomeTeam,AwayTeam,location,date,losing_abbr,...,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_win_percentage,home_wins,pace
2017-12-01-19-maryland,Purdue,Away,PURDUE,Purdue,0,MARYLAND,PURDUE,"Xfinity Center, College Park, Maryland","December 1, 2017",MARYLAND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-10-19-purdue,Purdue,Home,PURDUE,Purdue,1,PURDUE,IUPUI,"Mackey Arena, West Lafayette, Indiana","December 10, 2017",IUPUI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-16-12-purdue,Purdue,Home,PURDUE,Purdue,2,PURDUE,BUTLER,"Bankers Life Fieldhouse, Indianapolis, Indiana","December 16, 2017",BUTLER,...,30.600000,0.595800,13.520000,10.600000,32.600000,0.582400,18.600000,0.766600,8.400000,70.000000
2017-12-21-18-purdue,Purdue,Home,PURDUE,Purdue,3,PURDUE,TENNESSEE-STATE,"Mackey Arena, West Lafayette, Indiana","December 21, 2017",TENNESSEE-STATE,...,29.842105,0.624789,17.821053,14.105263,31.368421,0.590737,18.315789,0.804211,9.631579,72.889474
2017-12-03-16-purdue,Purdue,Home,PURDUE,Purdue,4,PURDUE,NORTHWESTERN,"Mackey Arena, West Lafayette, Indiana","December 3, 2017",NORTHWESTERN,...,30.738462,0.659769,15.278462,11.984615,31.630769,0.592092,18.600000,0.826138,10.615385,72.312308
2017-12-30-20-purdue,Purdue,Home,PURDUE,Purdue,5,PURDUE,LIPSCOMB,"Mackey Arena, West Lafayette, Indiana","December 30, 2017",LIPSCOMB,...,30.838863,0.627227,13.213744,10.071090,32.924171,0.583995,19.137441,0.816104,9.611374,68.890995
2017-12-07-18-purdue,Purdue,Home,PURDUE,Purdue,6,PURDUE,VALPARAISO,"Mackey Arena, West Lafayette, Indiana","December 7, 2017",VALPARAISO,...,37.475188,0.629337,12.952932,10.410526,35.509774,0.580708,20.548872,0.834702,10.849624,70.136692
2018-02-10-16-michigan-state,Purdue,Home,MICHIGAN-STATE,Michigan State,7,MICHIGAN-STATE,PURDUE,"Breslin Events Center, East Lansing, Michigan","February 10, 2018",PURDUE,...,36.952890,0.599477,12.261486,9.911122,37.453618,0.556382,20.708596,0.828789,10.194755,71.893978
2018-02-15-19-wisconsin,Purdue,Home,WISCONSIN,Wisconsin,8,WISCONSIN,PURDUE,"Kohl Center, Madison, Wisconsin","February 15, 2018",PURDUE,...,36.969231,0.579193,11.026122,8.554481,39.030611,0.553474,21.503410,0.849674,14.983347,69.086439
2018-02-18-20-purdue,Purdue,Home,PURDUE,Purdue,9,PURDUE,PENN-STATE,"Mackey Arena, West Lafayette, Indiana","February 18, 2018",PENN-STATE,...,36.295290,0.542166,11.735877,8.706953,35.940014,0.546467,19.619947,0.705704,13.962339,67.790585
